In [3]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class BreastCancerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Breast Cancer Prediction")

        # Create canvas and scrollbar for scrolling
        self.canvas = tk.Canvas(root)
        self.scrollbar = tk.Scrollbar(root, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = tk.Frame(self.canvas)

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        )

        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.canvas.pack(side="left", fill="both", expand=True)
        self.scrollbar.pack(side="right", fill="y")

        # Dataset and models
        self.df = None
        self.svm_model = None
        self.dt_model = None
        self.scaler = StandardScaler()

        # List of features
        self.columns = [
            "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
            "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean",
            "fractal_dimension_mean", "radius_se", "texture_se", "perimeter_se", "area_se",
            "smoothness_se", "compactness_se", "concavity_se", "concave points_se",
            "symmetry_se", "fractal_dimension_se", "radius_worst", "texture_worst",
            "perimeter_worst", "area_worst", "smoothness_worst", "compactness_worst",
            "concavity_worst", "concave points_worst", "symmetry_worst",
            "fractal_dimension_worst"
        ]

        # Layout
        self.create_widgets()

    def create_widgets(self):
        # Load dataset button
        tk.Button(self.scrollable_frame, text="Load Dataset", command=self.load_dataset).grid(row=0, column=0, padx=10, pady=10)

        # Train SVM button
        tk.Button(self.scrollable_frame, text="Train SVM", command=self.train_svm).grid(row=0, column=1, padx=10, pady=10)

        # Train Decision Tree button
        tk.Button(self.scrollable_frame, text="Train Decision Tree", command=self.train_dt).grid(row=0, column=2, padx=10, pady=10)

        # Prediction section
        tk.Label(self.scrollable_frame, text="Input Data for Prediction").grid(row=1, column=0, columnspan=3, pady=10)
        self.input_fields = []
        for i, col in enumerate(self.columns):
            tk.Label(self.scrollable_frame, text=col).grid(row=2+i, column=0, sticky='e', padx=5, pady=5)
            entry = tk.Entry(self.scrollable_frame)
            entry.grid(row=2+i, column=1, columnspan=2, padx=5, pady=5, sticky='w')
            self.input_fields.append(entry)

        # Predict button
        tk.Button(self.scrollable_frame, text="Predict", command=self.predict).grid(row=2+len(self.columns), column=0, columnspan=3, pady=10)

    def load_dataset(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
        if not file_path:
            return
        try:
            self.df = pd.read_csv(file_path)
            messagebox.showinfo("Success", "Dataset loaded successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load dataset: {e}")

    def train_svm(self):
        if self.df is None:
            messagebox.showerror("Error", "Please load a dataset first!")
            return
        try:
            X = self.df[self.columns]
            y = self.df['diagnosis'].map({'M': 1, 'B': 0})
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            self.scaler.fit(X_train)
            X_train_scaled = self.scaler.transform(X_train)
            X_test_scaled = self.scaler.transform(X_test)

            self.svm_model = SVC()
            self.svm_model.fit(X_train_scaled, y_train)

            y_pred = self.svm_model.predict(X_test_scaled)
            metrics = self.calculate_metrics(y_test, y_pred)
            self.display_metrics("SVM", metrics)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to train SVM: {e}")

    def train_dt(self):
        if self.df is None:
            messagebox.showerror("Error", "Please load a dataset first!")
            return
        try:
            X = self.df[self.columns]
            y = self.df['diagnosis'].map({'M': 1, 'B': 0})
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            self.dt_model = DecisionTreeClassifier()
            self.dt_model.fit(X_train, y_train)

            y_pred = self.dt_model.predict(X_test)
            metrics = self.calculate_metrics(y_test, y_pred)
            self.display_metrics("Decision Tree", metrics)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to train Decision Tree: {e}")

    def predict(self):
        if self.svm_model is None or self.dt_model is None:
            messagebox.showerror("Error", "Please train the models first!")
            return
        try:
            input_data = [float(entry.get()) for entry in self.input_fields]
            input_data_scaled = self.scaler.transform([input_data])

            svm_prediction = self.svm_model.predict(input_data_scaled)
            dt_prediction = self.dt_model.predict([input_data])

            messagebox.showinfo("Prediction", f"SVM Prediction: {svm_prediction[0]}\nDecision Tree Prediction: {dt_prediction[0]}")
        except Exception as e:
            messagebox.showerror("Error", f"Prediction failed: {e}")

    def calculate_metrics(self, y_true, y_pred):
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        return {"Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1}

    def display_metrics(self, model_name, metrics):
        metrics_str = "\n".join([f"{key}: {value:.2f}" for key, value in metrics.items()])
        messagebox.showinfo(f"{model_name} Metrics", metrics_str)

# Run the GUI
if __name__ == "__main__":
    root = tk.Tk()
    app = BreastCancerGUI(root)
    root.mainloop()
